<a href="https://colab.research.google.com/github/ShabibSaleh/DPAM-Nutrition/blob/main/Population_weighted_coverage_of_health_services.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read data from Google sheet

In [58]:
!pip install --upgrade gspread google-auth

In [59]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [60]:
# Open a sheet from a spreadsheet by name
spreadsheet = gc.open('DPAM Population-weighted coverage of health services')

# If you have the URL, you can do it this way:
# spreadsheet = gc.open_by_url('Your_Spreadsheet_URL')

# Data preparation and Cleaning

## on-track and off-track countries dataset

In [61]:
# Get a worksheet by name
worksheet = spreadsheet.worksheet('On & off-track countries')

# Get all values from the worksheet
rows = worksheet.get_all_values()




In [62]:
#Convert data to pandas DataFrame (if needed)
import pandas as pd
on_off_track_df = pd.DataFrame.from_records(rows[1:],columns=rows[0])  # Skip header row

In [63]:
# Checking for any missing values and the data types in the dataframe
on_off_track_df.info()

# Summary statistics to understand numerical columns, although most data seems categorical
on_off_track_df.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ISO3Code      200 non-null    object
 1   OfficialName  200 non-null    object
 2   Status.U5MR   200 non-null    object
dtypes: object(3)
memory usage: 4.8+ KB


,ISO3Code,OfficialName,Status.U5MR
count,200,200,200
unique,200,200,3
top,AFG,Afghanistan,Achieved
freq,1,1,134


In [64]:
# Step 1: Check for duplicates
duplicates = on_off_track_df.duplicated().sum()

# Step 2: Checking the unique values in the "Status.U5MR" column to ensure consistency
unique_status_values = on_off_track_df['Status.U5MR'].unique()

# Step 3: Renaming columns for clarity
on_off_track_df_cleaned = on_off_track_df.rename(columns={
    'ISO3Code': 'Country Code',
    'OfficialName': 'Country Name',
    'Status.U5MR': 'U5MR Status'
})

# Displaying the results
duplicates, unique_status_values, on_off_track_df_cleaned.head()


(0,
 array(['Acceleration Needed', 'Achieved', 'On Track'], dtype=object),
   Country Code Country Name          U5MR Status
 0          AFG  Afghanistan  Acceleration Needed
 1          AGO       Angola  Acceleration Needed
 2          AIA     Anguilla             Achieved
 3          ALB      Albania             Achieved
 4          AND      Andorra             Achieved)

In [65]:
# Updating the values in the 'U5MR Status' column
on_off_track_df_cleaned['U5MR Status'] = on_off_track_df_cleaned['U5MR Status'].replace({
    'Achieved': 'On-Track',
    'On Track': 'On-Track',
    'Acceleration Needed': 'Off-Track'
})

# Displaying the first few rows to confirm the changes
on_off_track_df_cleaned.head()


,Country Code,Country Name,U5MR Status
0,AFG,Afghanistan,Off-Track
1,AGO,Angola,Off-Track
2,AIA,Anguilla,On-Track
3,ALB,Albania,On-Track
4,AND,Andorra,On-Track


Extract on-track and off-track cleaned data countries dataset to new Google sheet

In [66]:
# Add a new sheet with a name and specify rows and columns
worksheet_title = 'on_off_track_df_cleaned'

ws = spreadsheet.worksheet(worksheet_title)

In [67]:
# Use the gspread_dataframe to set the DataFrame to the sheet
from gspread_dataframe import set_with_dataframe
# Set the DataFrame to the worksheet
set_with_dataframe(ws, on_off_track_df_cleaned)

## ANC4 & SAB Data

In [68]:
# Get a worksheet by name
ANC4_SAB_worksheet = spreadsheet.worksheet('ANC4 & SAB Data')

# Get all values from the worksheet
ANC4_SAB_rows = ANC4_SAB_worksheet.get_all_values()

In [69]:
# Checking the general information and structure of the ANC4 & SAB Data sheet
anc4_sab_df = pd.DataFrame.from_records(ANC4_SAB_rows[1:],columns=ANC4_SAB_rows[0])  # Skip header row
anc4_sab_df.info()

# Summary statistics for numerical columns
anc4_sab_df.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Geographic area  291 non-null    object
 1   Indicator        291 non-null    object
 2   Sex              291 non-null    object
 3   2022             291 non-null    object
 4   2021             291 non-null    object
 5   2020             291 non-null    object
 6   2019             291 non-null    object
 7   2018             291 non-null    object
dtypes: object(8)
memory usage: 18.3+ KB


,Geographic area,Indicator,Sex,2022,2021,2020,2019,2018
count,291,291,291,291,291,291,291,291
unique,181,2,1,62,34,54,71,84
top,Latin America and the Caribbean,Skilled birth attendant - percentage of delive...,Female,-,-,-,-,-
freq,7,159,291,210,251,213,175,159


In [70]:
# Converting the column names to strings
anc4_sab_df.columns = anc4_sab_df.columns.astype(str)


year_columns = ['2022', '2021', '2020', '2019', '2018']
anc4_sab_df[year_columns] = anc4_sab_df[year_columns].apply(pd.to_numeric, errors='coerce')

# Step 3: Drop the "Sex" column as it is redundant
anc4_sab_df_cleaned = anc4_sab_df.drop(columns=['Sex'])

# Displaying the cleaned data to verify
anc4_sab_df_cleaned.head()


,Geographic area,Indicator,2022,2021,2020,2019,2018
0,Afghanistan,Antenatal care 4+ visits - percentage of women...,NaN,NaN,27.6,NaN,20.9
1,Afghanistan,Skilled birth attendant - percentage of delive...,NaN,NaN,61.8,NaN,58.8
2,Africa,Antenatal care 4+ visits - percentage of women...,56.8,NaN,NaN,NaN,NaN
3,Africa,Skilled birth attendant - percentage of delive...,71.0,NaN,NaN,NaN,NaN
4,Albania,Antenatal care 4+ visits - percentage of women...,NaN,NaN,NaN,NaN,77.8


In [71]:
# Creating a new column 'Most Recent Coverage Estimate' which takes the most recent non-null value from 2018 to 2022
anc4_sab_df_cleaned['Most Recent Coverage Estimate'] = anc4_sab_df_cleaned[year_columns].bfill(axis=1).iloc[:, 0]

# Displaying the updated dataframe
anc4_sab_df_cleaned.head()


,Geographic area,Indicator,2022,2021,2020,2019,2018,Most Recent Coverage Estimate
0,Afghanistan,Antenatal care 4+ visits - percentage of women...,NaN,NaN,27.6,NaN,20.9,27.6
1,Afghanistan,Skilled birth attendant - percentage of delive...,NaN,NaN,61.8,NaN,58.8,61.8
2,Africa,Antenatal care 4+ visits - percentage of women...,56.8,NaN,NaN,NaN,NaN,56.8
3,Africa,Skilled birth attendant - percentage of delive...,71.0,NaN,NaN,NaN,NaN,71.0
4,Albania,Antenatal care 4+ visits - percentage of women...,NaN,NaN,NaN,NaN,77.8,77.8


In [72]:
# Dropping the year columns as requested
anc4_sab_df_cleaned = anc4_sab_df_cleaned.drop(columns=year_columns)

# Displaying the cleaned dataframe with the year columns removed
anc4_sab_df_cleaned.head()


,Geographic area,Indicator,Most Recent Coverage Estimate
0,Afghanistan,Antenatal care 4+ visits - percentage of women...,27.6
1,Afghanistan,Skilled birth attendant - percentage of delive...,61.8
2,Africa,Antenatal care 4+ visits - percentage of women...,56.8
3,Africa,Skilled birth attendant - percentage of delive...,71.0
4,Albania,Antenatal care 4+ visits - percentage of women...,77.8


In [73]:
# Replacing the values in the 'Indicator' column as requested
anc4_sab_df_cleaned['Indicator'] = anc4_sab_df_cleaned['Indicator'].replace({
    'Antenatal care 4+ visits - percentage of women (aged 15-49 years) attended at least four times during pregnancy by any provider': 'ANC4',
    'Skilled birth attendant - percentage of deliveries attended by skilled health personnel': 'SAB'
})

# Displaying the updated dataframe to confirm changes
anc4_sab_df_cleaned.head()


,Geographic area,Indicator,Most Recent Coverage Estimate
0,Afghanistan,ANC4,27.6
1,Afghanistan,SAB,61.8
2,Africa,ANC4,56.8
3,Africa,SAB,71.0
4,Albania,ANC4,77.8


In [74]:
# Creating a pivot table with 'Geographic area' as the index, 'Indicator' as columns, and summing 'Most Recent Coverage Estimate'
pivot_table = anc4_sab_df_cleaned.pivot_table(
    index='Geographic area',
    columns='Indicator',
    values='Most Recent Coverage Estimate',
    aggfunc='sum'
).reset_index()

# Displaying the resulting pivot table
pivot_table.head()


Indicator,Geographic area,ANC4,SAB
0,Afghanistan,27.6,61.8
1,Africa,56.8,71.0
2,Albania,77.8,99.8
3,Algeria,69.8,98.8
4,Americas,94.0,96.8


In [75]:
# Renaming the 'Geographic area' column to 'Country Name'
pivot_table.rename(columns={'Geographic area': 'Country Name'}, inplace=True)

# Displaying the updated pivot table
pivot_table.head()


Indicator,Country Name,ANC4,SAB
0,Afghanistan,27.6,61.8
1,Africa,56.8,71.0
2,Albania,77.8,99.8
3,Algeria,69.8,98.8
4,Americas,94.0,96.8


Extract ANC4 & SAB cleaned dataset to new Google sheet

In [76]:
# Add a new sheet with a name and specify rows and columns
worksheet_title = 'anc4_sab_df_cleaned'

ws = spreadsheet.worksheet(worksheet_title)

In [77]:
# Use the gspread_dataframe to set the DataFrame to the sheet
from gspread_dataframe import set_with_dataframe
# Set the DataFrame to the worksheet
set_with_dataframe(ws, pivot_table)

## Population data

In [80]:
# Get a worksheet by name
population_worksheet = spreadsheet.worksheet('Population data')

# Get all values from the worksheet
population_rows = population_worksheet.get_all_values()

In [81]:
# Checking the general information and structure of the ANC4 & SAB Data sheet
population_df = pd.DataFrame.from_records(population_rows[1:],columns=population_rows[0])  # Skip header row
population_df.info()

# Summary statistics for numerical columns
population_df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271 entries, 0 to 270
Data columns (total 65 columns):
 #   Column                                                                                          Non-Null Count  Dtype 
---  ------                                                                                          --------------  ----- 
 0   Index                                                                                           271 non-null    object
 1   Variant                                                                                         271 non-null    object
 2   Region, subregion, country or area *                                                            271 non-null    object
 3   Notes                                                                                           271 non-null    object
 4   Location code                                                                                   271 non-null    object
 5   ISO3 Alpha-code       

,Index,Variant,"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,...,"Male Mortality before Age 60 (deaths under age 60 per 1,000 male live births)","Female Mortality before Age 60 (deaths under age 60 per 1,000 female live births)","Mortality between Age 15 and 50, both sexes (deaths under age 50 per 1,000 alive at age 15)","Male Mortality between Age 15 and 50 (deaths under age 50 per 1,000 males alive at age 15)","Female Mortality between Age 15 and 50 (deaths under age 50 per 1,000 females alive at age 15)","Mortality between Age 15 and 60, both sexes (deaths under age 60 per 1,000 alive at age 15)","Male Mortality between Age 15 and 60 (deaths under age 60 per 1,000 males alive at age 15)","Female Mortality between Age 15 and 60 (deaths under age 60 per 1,000 females alive at age 15)",Net Number of Migrants (thousands),"Net Migration Rate (per 1,000 population)"
count,271,271,271,271,271,271,271,271,271,271,...,271,271,271,271,271,271,271,271,271,271
unique,271,1,271,43,271,238,238,268,8,31,...,197,176,137,150,119,179,183,162,112,108
top,1,Medium,WORLD,,900,,,,Country/Area,915,...,183,47,41,26,21,109,228,40,0,0
freq,1,271,1,196,1,34,34,4,237,28,...,4,5,6,7,9,6,5,6,58,30


In [82]:
# Selecting relevant columns
relevant_columns = [
    'Region, subregion, country or area *',
    'ISO3 Alpha-code',
    'Year',
    'Total Population, as of 1 July (thousands)',
    'Male Population, as of 1 July (thousands)',
    'Female Population, as of 1 July (thousands)'
]

population_df_cleaned = population_df[relevant_columns]

# Renaming columns for clarity
population_df_cleaned.columns = [
    'Country Name',
    'ISO3 Code',
    'Year',
    'Total Population (thousands)',
    'Male Population (thousands)',
    'Female Population (thousands)'
]

# Filtering out rows where 'Type' is not 'Country/Area' (assuming countries are our focus)
population_df_cleaned = population_df_cleaned.dropna(subset=['ISO3 Code'])

# Displaying the first few rows of the cleaned data
population_df_cleaned.head()


,Country Name,ISO3 Code,Year,Total Population (thousands),Male Population (thousands),Female Population (thousands)
0,WORLD,,2022,7 975 105,4 008 580,3 966 526
1,Sub-Saharan Africa,,2022,1 166 766,581 308,585 458
2,Northern Africa and Western Asia,,2022,553 690,284 437,269 254
3,Central and Southern Asia,,2022,2 084 590,1 064 471,1 020 119
4,Eastern and South-Eastern Asia,,2022,2 344 325,1 185 075,1 159 250
